# FastAPI

## Lesson Materials

For this lesson, two libraries are required:
- `fastapi` allows us to define the API. See the docs [here](https://fastapi.tiangolo.com).
- `uvicorn` allows FastAPI to handle requests in parallel. See the docs [here](https://www.uvicorn.org).

In [ ]:
pip install fastapi
pip install uvicorn

# Introduction
According to the official website, FastAPI is a modern, fast (high-performance), web framework for building application programming interfaces (APIs) with Python 3.6+.

The advantages of FastAPI relative to other web frameworks include the following:

1. __Easy to use__: The code structure and its encapsulation make it very intuitive to find methods quickly.
2. __Fast to code__: FastAPI performs most of the laborious coding tasks. The user simply adds the type of request to be expected and the corresponding response.
3. __Employs both OpenAPI and JSON__: The browser displays the data in an organised and readable format, and users can retrieve the data efficiently.
4. __Queries are considerably easy to format__: By default, queries have the `str` format; however, if a specific type hint is provided in the function, the input will be cast to that type.
5. __Processes requests asynchronously__: Multiple requests can be sent to a FastAPI server while it is processing the previous request. Therefore, new requests can be sent without waiting for the completion of the previous request.
6. __Integrates dataclass decorators__: This makes it significantly easy to get an application up and running.
7. __Integrates pytest__: APIs can be easily tested before being deployed.

Although FastAPI is relatively new, compared to Flask or Django, its adoption is increasing rapidly. We encourage you to explore other libraries for creating applications, such as Tornado and Hug. However, in this lesson, we will focus on FastAPI because of its power and simplicity. We will create APIs with minimal functionality.

## Building APIs

Building an API using FastAPI is incredibly easy. For this, a decorator is added to the function that will be run in an application:

In [1]:
import fastapi
import uvicorn

### Address structure
As a reminder, the address structure of an API is as follows:
```
REQUEST = <ROOT_URL>/<Path>?<Query Parameters>
```

Within `Query Parameters`, the parameters are separated by the ampersand symbol, `&`. See the example below:

<div style="text-align:center"><img src="images/API_Structure.png" width=600/></div>


### Example 1
Now, we create the first instance of a FastAPI.

This instance has methods that can be employed to decorate functions with functionality, e.g. defining a function that runs on a `GET` request to a particular server.

> If the API is run within the notebook `uvicorn`, it will run indefinitely, and you will not be able to run the next cells. Thus, the following code should be run in a `.py` file.

In [1]:
%%python3
import fastapi
import uvicorn
api = fastapi.FastAPI() 
 
# The endpoint will be our localhost (127.0.0.1),
# and the listening port will be 8000.
# The path is '/test/calculate'; however, you can use any path of your choosing.
# Simply ensure that you use the same path in your browser to check the results.

@api.get('/test/calculate')
def calculate():
    return 2 + 2

uvicorn.run(api, port=8000, host='127.0.0.1') # use uvicorn to run the api so that it processes requests asynchronously

Process is interrupted.


After running the script, you can examine the API output by visiting the path, `/test/calculate`. To do this, open [127.0.0.1:8000/test/calculate](127.0.0.1:8000/test/calculate) in a browser. The output should be similar to that in the figure below (depending on the browser):

<div style="text-align:center"><img src="images/FastAPI_1.png" width=600/></div>



The API returns a simple number, which is not useful. Next, we add some query parameters by simply introducing arguments into the decorated function.

Query-string params are passed into a request following the '`?`' symbol (after the path), as key-value pairs on either side of an '`=`' sign separated by `&`. For example, `my/path?firstparam=1&secondparam=3`.

FastAPI expects the query-string names to match the function argument names.

In [ ]:
%%python3
import fastapi
import uvicorn
api = fastapi.FastAPI() 

@api.get('/test/calculate')
def calculate(x, y):
    return x + y

uvicorn.run(api, port=8000, host='127.0.0.1')

Upon visiting [127.0.0.1:8000/test/calculate](127.0.0.1:8000/test/calculate) once more, the API returns a response complaining about the lack of sufficient arguments:

<div style="text-align:center"><img src="images/FastAPI_2.png" width=600/></div>

As mentioned, `x` and `y` are parameters that must be passed if we are running a function. They can be assigned default values and/or passed to the URL in the form of a query:

<div style="text-align:center"><img src="images/FastAPI_3.png" width=400/></div>

As can be observed, the API returns "15", even though 1 + 5 is not 15. This is because the parameters of the query are read as strings by default. 

> Type hinting can be applied to change the default type of parameters.

In [ ]:
import fastapi
import uvicorn
api = fastapi.FastAPI() 
 
@api.get('/test/calculate')
def calculate(x: int, y: int): # integer type hints define the default type of the arguments.
    return x + y

uvicorn.run(api, port=8000, host='127.0.0.1')

Moving forward, we add responses characteristic of an API. Instead of addition, we divide `x` by `y`: 

In [ ]:
import fastapi
import uvicorn
api = fastapi.FastAPI() 
 
@api.get('/test/calculate')
def calculate(x: int, y: int):
    return x / y

uvicorn.run(api, port=8000, host='127.0.0.1')

The code works; however, if we use `y=0`, Python throws the error, Error: 500 `Internal Server Error`.

<div style="text-align:center"><img src="images/FastAPI_4.png" width=400/></div>

Although the error message informs the user of the origin of the issue, it does not provide much information. To provide more information, we could send a 400 Error: `Bad Request`, indicating that the given parameters are invalid, along with a message. All of this will be encapsulated in the Response class from the FastAPI library.

In [ ]:
import fastapi
import uvicorn
api = fastapi.FastAPI() 
 
@api.get('/test/calculate')
def calculate(x: int, y: int):
    if y == 0:
        return fastapi.Response(content='y cannot be zero', status_code=400)
    return x / y

uvicorn.run(api, port=8000, host='127.0.0.1')

Moreover, the data type to be returned can be added to the response. In many cases, APIs will return a JSON file; thus, we can specify the error message in the response returned to be in JSON format.

In [ ]:
import fastapi
import uvicorn
import json
api = fastapi.FastAPI() 
 
@api.get('/test/calculate')
def calculate(x: int, y: int):
    if y == 0:
        return fastapi.Response(content=json.dumps({"error" : "y cannot be zero"}),
                                media_type='application/json',
                                status_code=400)
    return x / y

uvicorn.run(api, port=8000, host='127.0.0.1')

Thus, the response will contain a header, indicating that it is in JSON format.

<div style="text-align:center"><img src="images/FastAPI_5.png" width=400/></div>

### Post Requests

It is also simple to create `POST` method integrations with FastAPI using the `.post()` decorator.


In [ ]:
import fastapi
import uvicorn
import json
api = fastapi.FastAPI() 
 
@api.post('/send/data')
def process_data(x: int, y: int):
    
    if y == 0:
        return fastapi.Response(content=json.dumps({"error" : "y cannot be zero"}),
                                media_type='application/json',
                                status_code=400)
    return x / y

uvicorn.run(api, port=8000, host='127.0.0.1')

Functionally, post requests do not require a response; however, as shown in the table [here](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods/POST), a `body` key should always be returned in the response. If the request inserts some data into a database entry, this `body` key typically returns the resulting database entry.

### Example 2 

The second API will accept the name of a celebrity and return the corresponding data of birth (DOB). Similar to the steps in the last example, we create a very simple API and build on it. Once more, we will employ the magic cell, although this is not recommended. Ensure that the virtual environment where you installed fastapi and uvicorn is activated.


In [ ]:
%%python
import fastapi
import uvicorn
api = fastapi.FastAPI() 

@api.get('/')
def index():
    return 'Welcome to the celebrity DOB API!'

if __name__ == '__main__':
    uvicorn.run(api, port=8000, host='127.0.0.1')

## CORS

Commonly, requests are made to an API from a website. This website, which has a different URL, is known to be of a different 'origin'. By default, cross-origin requests may not be allowed. This situation is known as Cross-Origin Resource Sharing (CORS). Read about how to implement CORS [here](https://fastapi.tiangolo.com/tutorial/cors/).

## Conclusion
At this point, you should have a good understanding of

- FastAPI and its advantages.
- how to build APIs.
- the address structure of an API.
- how to create POST method integrations.
- cross-origin resource sharing.


## Further Reading

FastAPI generates documentation automatically. To access the docs, you simply run your API and go to the `/docs` resources, where you will find the API documentation. The details are provided [here](https://fastapi.tiangolo.com/tutorial/body/#automatic-docs).